https://github.com/pytorch/pytorch/pull/2016

In [0]:
!pip3 install Pillow==4.0.0
!pip3 install torch==0.4.0
!pip3 install torchvision==0.2.1

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.optim.lr_scheduler import LambdaLR, StepLR, MultiStepLR, ExponentialLR, CosineAnnealingLR, ReduceLROnPlateau

import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

epochs = 20
batch_size = 100
log_interval = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#モデルの定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

#実行クラス
class Runner():
    def __init__(self, model):
        self.model = model
        self.scheduler = None

    def train(self, model, device, train_loader, optimizer, epoch, use_scheduler=False):
        model.train()
        train_loss_list = []
        if use_scheduler == True:
            self.scheduler.step()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad() #勾配クリア
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()  #誤差逆伝播の計算
            optimizer.step() #重みの更新
            
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Lr：{} '.format(
                    epoch, batch_idx * len(data), 
                    len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), 
                    loss.item(), #lossはloss.item()
                    optimizer.param_groups[0]["lr"]
                ))
                train_loss_list.append(loss.item())
        return train_loss_list

    def test(self, model, device, test_loader):
        model.eval()
        test_loss = 0
        correct = 0
        test_loss_list = []
        correct_list = []
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
                pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, 
            correct, 
            len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)
        ))
        correct_list.append(100. * correct / len(test_loader.dataset))
        test_loss_list.append(test_loss)
        return test_loss_list, correct_list

In [0]:

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=200)

In [0]:
def cyclical_lr(step_sz, min_lr=0.001, max_lr=1, mode='triangular', scale_func=None, scale_md='cycles', gamma=1.):
    if scale_func == None:
        if mode == 'triangular':
            scale_fn = lambda x: 1.
            scale_mode = 'cycles'
        elif mode == 'triangular2':
            scale_fn = lambda x: 1 / (2.**(x - 1))
            scale_mode = 'cycles'
        elif mode == 'exp_range':
            scale_fn = lambda x: gamma**(x)
            scale_mode = 'iterations'
        else:
            raise ValueError(f'The {mode} is not valid value!')
    else:
        scale_fn = scale_func
        scale_mode = scale_md

    lr_lambda = lambda iters: min_lr + (max_lr - min_lr) * rel_val(iters, step_sz, scale_mode)

    def rel_val(iteration, stepsize, mode):
        cycle = math.floor(1 + iteration / (2 * stepsize))
        x = abs(iteration / stepsize - 2 * cycle + 1)
        if mode == 'cycles':
            return max(0, (1 - x)) * scale_fn(cycle)
        elif mode == 'iterations':
            return max(0, (1 - x)) * scale_fn(iteration)
        else:
            raise ValueError(f'The {scale_mode} is not valid value!')

    return lr_lambda

In [14]:
model = Net()
model.cuda()

train_loss_list = []
test_loss_list = []
correct_list = []

runner = Runner(model)

# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
optimizer = optim.SGD(model.parameters(), lr=1.)
step_size = 2*len(train_loader)
# clr = cyclical_lr(step_size, min_lr=0.001, max_lr=1, mode='triangular2')
clr = cyclical_lr(step_size, min_lr=0.001, max_lr=0.005)
scheduler = LambdaLR(optimizer, [clr])
runner.scheduler = scheduler


def main(optimizer, model):
    for epoch in range(1, epochs + 1):
        ref_loss = runner.train(model, device, train_loader, optimizer, epoch)
        train_loss_list.extend(ref_loss)
        ref_loss, ref_correct = runner.test(model, device, test_loader)
        test_loss_list.extend(ref_loss)
        correct_list.extend(ref_correct)
    return train_loss_list, test_loss_list, correct_list
        
train_loss_list, test_loss_list, correct_list = main(optimizer, model)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.326461	 Lr：0.001 
Train Epoch: 1 [1000/60000 (2%)]	Loss: 2.321351	 Lr：0.001 
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.311539	 Lr：0.001 
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.320342	 Lr：0.001 
Train Epoch: 1 [4000/60000 (7%)]	Loss: 2.327782	 Lr：0.001 
Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.323558	 Lr：0.001 
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.321137	 Lr：0.001 
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.311230	 Lr：0.001 
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.317993	 Lr：0.001 
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.328308	 Lr：0.001 
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.302110	 Lr：0.001 
Train Epoch: 1 [11000/60000 (18%)]	Loss: 2.312165	 Lr：0.001 
Train Epoch: 1 [12000/60000 (20%)]	Loss: 2.328779	 Lr：0.001 
Train Epoch: 1 [13000/60000 (22%)]	Loss: 2.318507	 Lr：0.001 
Train Epoch: 1 [14000/60000 (23%)]	Loss: 2.334375	 Lr：0.001 
Train Epoch: 1 [15000/60000 (25%)]	Loss: 2.310396	 Lr：0.001 
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.

KeyboardInterrupt: ignored